[View in Colaboratory](https://colab.research.google.com/github/tcglarry/kaggle_predict_sales/blob/master/sales_first_try.ipynb)

In [1]:


# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:

!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb  PiXNet				      test for excel
aoi_test	     Sample upload.txt			      Untitled0.ipynb
Colab Notebooks      TAHU PowerPoint Template.pdf	      Untitled1.ipynb
kaggle		     TAHU PowerPoint Template - 柱狀圖 1.ods


In [10]:
cd drive/kaggle/predict_sales

/content/drive/kaggle/predict_sales


In [43]:
!ls

data.zip	     predict_sales_first_try.h5  shops.csv
item_categories.csv  sales_first_try.ipynb	 test.csv
items.csv	     sales_train.csv		 train_data.npy
label_data.npy	     sales_train.csv.gz		 Untitled0.ipynb
prediction.npy	     sample_submission.csv	 Untitled.ipynb


In [12]:
!unzip data.zip

Archive:  data.zip
  inflating: train_data.npy          
  inflating: label_data.npy          


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Conv2DTranspose
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers

Using TensorFlow backend.


In [0]:
train_df = pd.read_csv('sales_train.csv')

In [0]:
items = pd.read_csv('items.csv')
item_cat = pd.read_csv('item_categories.csv')

In [27]:
item_list = list(set(train_df['item_id'].tolist()))
len(list(item_list)) > len(set(item_list))

False

In [28]:
len(item_list)

21807

In [29]:

shop_list = list(set(train_df['shop_id'].tolist()))
print (len(list(shop_list)) > len(set(shop_list)))
len(shop_list)

False


60

In [30]:
train_df['date_block_num'].max()

33

In [32]:
np_train = []

for i in range(34):
    grand_df = pd.DataFrame(0,index=item_list, columns=shop_list)
    try_df = train_df.loc[train_df['date_block_num']== i ,:]
    try_df = try_df.pivot_table(index='item_id', columns='shop_id', values='item_cnt_day', aggfunc = np.sum, fill_value=0)
 
    grand_df.update(try_df)
    
    #print ('i=', i, 'sum=', grand_df.sum().sum())
    month_values= grand_df.values
    month_values=month_values[np.newaxis,:,:]
    np_train.append(month_values)

np_train = np.concatenate(np_train, axis=0)
np_train.shape

(34, 21807, 60)

In [0]:
def shuffle_cnn(length,window,original):
    train_list = []
    label_list = []
    b_s = int(np.floor(length/window))

    for i in range(length-window):

        train_list.append(original[i:i+window,:,:])
        label_list.append(original[i+window,:,:])
    return train_list, label_list



In [0]:
window = 5 
length = 34
train_list, label_list = shuffle_cnn(length,window,np_train)



In [0]:
train_data = np.stack(train_list)
train_data=train_data.transpose(0,2,3,1)
label_data = np.stack(label_list)
label_data = label_data[:,:,:,np.newaxis]
print (train_data.shape)
print (label_data.shape)

(29, 21807, 60, 5)
(29, 21807, 60, 1)


In [0]:

np.save('train_data.npy',train_data)
np.save('label_data.npy', label_data)

# Starts from here if train data ready
===

In [0]:
train_data = np.load('train_data.npy')
label_data = np.load('label_data.npy')

In [15]:

train_x,val_x,train_y,val_y = train_test_split(train_data,label_data, test_size = 0.2, shuffle=True)

print (train_x.shape)
print (train_y.shape)






(23, 21807, 60, 5)
(23, 21807, 60, 1)


In [0]:

def conv_block(ch, activation= 'relu', padding='same',kernel_regularizer=regularizers.l2(0.01)):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )


In [0]:
input_size = (21807,60,5)
def build_model(ch=4): 
    inputs = Input(shape= input_size)
    #reshape_inputs = Permute((2,3,1),input_shape=(5,21807,60))(inputs)
    x = Conv2D(32,(3,3),padding='same',activation='relu')(inputs)
    x = Conv2D(32,(3,3),padding='same',activation='relu')(x)
    x = Conv2D(256,(364,1),strides=(364,1),padding='same',activation='relu')(x)
    
    
    conv1 =  conv_block(ch)(x)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    print ('pool1',pool1.get_shape())
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    print ('poo12',pool2.get_shape())
    
    conv_test = Conv2D(64,(2,2),padding='valid')(pool2)
    print ('conv_test',conv_test.get_shape())
    
    
    
    conv3 =  conv_block(ch*4)(pool2)
    conv3 = conv_block(ch*4)(conv3)
    print ('conv3',conv3.get_shape())
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    print ('poo13',pool3.get_shape())
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4_1 = Conv2D(ch*8,(2,2),padding='valid',activation='relu')(conv4)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    print ('poo14',pool4.get_shape())
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)
    print ('conv5',conv5.get_shape())
    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4_1], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    print ('conv6',conv6.get_shape())
    
    up7 = UpSampling2D(size=(3,3))(conv6)
    up7_1=  Conv2D(ch*8,(4,4),padding='valid',activation='relu')(up7)
    print ('up7_1',up7_1.get_shape())
    up7_2 = concatenate ([up7_1, conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7_2)
    conv7 = conv_block(ch*4)(conv7)
    print ('conv7',conv7.get_shape())
    
    conv7 = Conv2DTranspose(ch*4,(3,3),strides=(4,4),padding='valid')(conv7)
    print ('conv7',conv7.get_shape())
    
    
    conv8 = Conv2DTranspose(32,(363,1),strides=(363,1),padding='same',activation='relu')(conv7)
    conv9 = Conv2DTranspose(32,(28,1),strides=(1,1),padding='valid',activation='relu')(conv8)
    outputs = Conv2D(1,(1,1),strides=(1,1),padding='valid',activation='relu')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    return model

    

In [18]:
model = build_model()

pool1 (?, 30, 30, 4)
poo12 (?, 15, 15, 8)
conv_test (?, 14, 14, 64)
conv3 (?, 15, 15, 16)
poo13 (?, 7, 7, 16)
poo14 (?, 3, 3, 32)
conv5 (?, 3, 3, 64)
conv6 (?, 6, 6, 32)
up7_1 (?, 15, 15, 32)
conv7 (?, 15, 15, 16)
conv7 (?, ?, ?, 16)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 21807, 60, 5) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 21807, 60, 32 1472        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 21807, 60, 32 9248        conv2d_1[0][0]                   
_________________________________________________________________________

In [0]:
model.compile(optimizer='Adam',loss='mse')




In [21]:
!pwd

/content/drive/kaggle/predict_sales


In [22]:
src=''
checkpoint = ModelCheckpoint(filepath=src+'predict_sales_first_try.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=10)
callback_list = [checkpoint, earlystop]
model.fit(x=train_x, y=train_y, batch_size=4, epochs=300, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)


Train on 18 samples, validate on 5 samples
Epoch 1/300
18/18 [==============================] - 9s 477ms/step - loss: 3.4132 - val_loss: 1.5787
Epoch 2/300
18/18 [==============================] - 8s 472ms/step - loss: 3.3732 - val_loss: 1.3992
Epoch 3/300
18/18 [==============================] - 8s 466ms/step - loss: 3.0457 - val_loss: 1.4944
Epoch 4/300
18/18 [==============================] - 8s 460ms/step - loss: 3.0983 - val_loss: 1.3163
Epoch 5/300
18/18 [==============================] - 8s 471ms/step - loss: 2.9412 - val_loss: 1.4692
Epoch 6/300
18/18 [==============================] - 8s 459ms/step - loss: 2.9188 - val_loss: 1.2711
Epoch 7/300
18/18 [==============================] - 8s 463ms/step - loss: 2.8717 - val_loss: 1.3044
Epoch 8/300
18/18 [==============================] - 8s 460ms/step - loss: 2.7861 - val_loss: 1.2238
Epoch 9/300
18/18 [==============================] - 8s 471ms/step - loss: 2.7686 - val_loss: 1.2685
Epoch 10/300
18/18 [============================

In [23]:
model.evaluate(val_x,val_y)

6/6 [==============================] - 4s 604ms/step


3.013324499130249

In [24]:
test_data = train_data[:,:,:,-5:]
test_data.shape

(29, 21807, 60, 5)

In [35]:
test_data = np_train[-5:,:,:]
test_data = test_data.transpose(1,2,0)
test_data = test_data[np.newaxis,:,:,:]
test_data.shape

(1, 21807, 60, 5)

In [0]:
prediction = model.predict(test_data)

In [38]:
prediction = np.squeeze(prediction)
prediction.shape

(21807, 60)

In [0]:
np.save('prediction.npy',prediction)

In [0]:
import pandas as pd
import numpy as np
prediction = np.load('drive/kaggle/predict_sales/prediction.npy')

In [0]:

test_df = pd.read_csv('drive/kaggle/predict_sales/test2.csv')
sample_df = pd.read_csv('drive/kaggle/predict_sales/sample_submission.csv')
prediction_df = pd.DataFrame(0,index=test_df['item_id'],columns=test_df['shop_id'])

adc.json  drive  sample_data
